In [2]:
import pandas as pd
import numpy as np
from yahoo_fin.stock_info import get_data
import statsmodels.formula.api as smf
from pypfopt.efficient_frontier import EfficientFrontier
import datetime
import requests 
from bs4 import BeautifulSoup
from scrapy import Selector
import json
import time

In [3]:
START = TRAIN_START = datetime.datetime(2018,1,1)
TEST_START = datetime.datetime(2020,1,5)
END = datetime.datetime(2020,12,31)

In [4]:
CC100_Index = pd.read_csv("indexReturns-BITW100.csv")
CC100_Index['date'] = pd.to_datetime(CC100_Index['date']).dt.tz_localize(None)
CC100_Index = CC100_Index.sort_values('date').set_index('date')
CC100_Index = CC100_Index.truncate(before = START,after = END)
CC100_Index = CC100_Index.resample('W').last()[:-1]
CC100_Return = np.log(CC100_Index/ CC100_Index.shift(1)).fillna(axis = 0, method ='bfill').rename(columns = {"MarketIndex":"MarketReturn"})
CC100 = pd.concat([CC100_Index, CC100_Return],axis =1)

In [5]:
CC100

,MarketIndex,MarketReturn
date,,
2018-01-07,34027.35,-0.108315
2018-01-14,30534.28,-0.108315
2018-01-21,25761.58,-0.169966
2018-01-28,22724.66,-0.125434
2018-02-04,18134.05,-0.225659
...,...,...
2020-11-29,18386.98,-0.001030
2020-12-06,19257.57,0.046261
2020-12-13,19002.90,-0.013313


In [7]:
CC_string = "ADA,ADX,AE,ANT,ARDR,ARK,BAT,BCH,BCN,BNB,BNT,BTC,BTG,BTS,CVC,DASH,DCR,DGB,DGD,DNT,DOGE,EDG,EOS,ETC,ETH,FUN,GAS,GBYTE,GNO,HC,ICX,KIN,KMD,KNC,LINK,LRC,LSK,LTC,MAID,MCO,MIOTA,MLN,MONA,MTL,NANO,NEO,NXS,OMG,PIVX,PPT,QRL,QTUM,REP,RLC,SALT,SC,SNT,STEEM,STORJ,SYS,TRX,VERI,WAVES,WTC,XEM,XLM,XMR,XRP,XVG,ZEC,ZEN,ZRX"
CC_temp_list = CC_string.split(",")
CC_list = [i+'-USD' for i in CC_temp_list]

# Fetch and save data of all 72 CC from yahoo finance

In [10]:
data_all = pd.DataFrame()
for i in CC_list:
    print(i)
    data = get_data(i,start_date = START,end_date = END).resample('W').last()[:-1]
    data_all = pd.concat([data_all,data])
data_all.index.names = ['date']
data_all.to_csv('CC_data.csv')

ADA-USD


KeyboardInterrupt: 

# Read data of all 72 CC 

In [11]:
data_copy = pd.read_csv('CC_data.csv')
data_copy['ticker'] = data_copy['ticker'].str.replace('-USD',' ')
data_copy = data_copy.copy().reset_index().set_index(['ticker','date'])

In [12]:
data_copy

index      open      high       low     close  adjclose  \
ticker date                                                                  
ADA    2018-01-07      0  1.024150  1.043960  0.991391  1.008520  1.008520   
       2018-01-14      1  0.881961  0.881961  0.749514  0.802262  0.802262   
       2018-01-21      2  0.706391  0.706391  0.590793  0.613163  0.613163   
       2018-01-28      3  0.620560  0.681946  0.620560  0.670286  0.670286   
       2018-02-04      4  0.433865  0.480140  0.369823  0.379303  0.379303   
...                  ...       ...       ...       ...       ...       ...   
ZRX    2020-11-29  11227  0.415439  0.419485  0.405365  0.414702  0.414702   
       2020-12-06  11228  0.407349  0.410595  0.394628  0.400748  0.400748   
       2020-12-13  11229  0.392727  0.403993  0.386825  0.396126  0.396126   
       2020-12-20  11230  0.424108  0.424108  0.390759  0.395337  0.395337   
       2020-12-27  11231  0.354252  0.424531  0.354252  0.373358  0.373358   

                        volume  
ticker date                     
ADA    2018-01-07  226067008.0  
       2018-01-14  160712000.0  
       2018-01-21  847731008.0  
       2018-01-28  325145984.0  
       2018-02-04  814753984.0  
...                        ...  
ZRX    2020-11-29   37263535.0  
       2020-12-06   33594446.0  
       2020-12-13   31834561.0  
       2020-12-20   45781167.0  
       2020-12-27  142214520.0  

[11232 rows x 7 columns]

# Save weekly top 10 market cap CC

In [13]:
def top10_Market(date):
    date = date.replace('-', '')
    url = 'https://coinmarketcap.com/historical/' + date
    cmc = requests.get(url)
    soup = BeautifulSoup(cmc.content, 'html.parser')
    data = soup.find('script', id='__NEXT_DATA__',type='application/json')
    coin_data = json.loads(data.contents[0])
    listings = coin_data['props']['initialState']['cryptocurrency']['listingHistorical']['data']
    df = pd.DataFrame(listings)
    df = df[['symbol']]
    CC_string = "ADA,ADX,AE,ANT,ARDR,ARK,BAT,BCH,BCN,BNB,BNT,BTC,BTG,BTS,CVC,DASH,DCR,DGB,DGD,DNT,DOGE,EDG,EOS,ETC,ETH,FUN,GAS,GBYTE,GNO,HC,ICX,KIN,KMD,KNC,LINK,LRC,LSK,LTC,MAID,MCO,MIOTA,MLN,MONA,MTL,NANO,NEO,NXS,OMG,PIVX,PPT,QRL,QTUM,REP,RLC,SALT,SC,SNT,STEEM,STORJ,SYS,TRX,VERI,WAVES,WTC,XEM,XLM,XMR,XRP,XVG,ZEC,ZEN,ZRX"
    CC_temp_list = CC_string.split(",")
    df = df[df['symbol'].isin(CC_temp_list)] 
    return df['symbol']

In [25]:
price_top10_Market = pd.DataFrame(columns = ['Date', 'top10_Market'])

i = 1

for date in CC100.index:
    print(date)
    date = str(date).split(' ')[0]
#     print(top10_Market(date))
    CC_list_temp = top10_Market(date).iloc[:10]
    Append = {'Date': date, 'top10_Market' : CC_list_temp.tolist()}
    price_top10_Market = price_top10_Market.append(Append, ignore_index=True)

    i = i+1
    if i == 5:
        i = 0
        time.sleep(120)


2018-01-07 00:00:00
2018-01-14 00:00:00
2018-01-21 00:00:00
2018-01-28 00:00:00
2018-02-04 00:00:00
2018-02-11 00:00:00
2018-02-18 00:00:00
2018-02-25 00:00:00
2018-03-04 00:00:00
2018-03-11 00:00:00
2018-03-18 00:00:00
2018-03-25 00:00:00
2018-04-01 00:00:00
2018-04-08 00:00:00
2018-04-15 00:00:00
2018-04-22 00:00:00
2018-04-29 00:00:00
2018-05-06 00:00:00
2018-05-13 00:00:00
2018-05-20 00:00:00
2018-05-27 00:00:00
2018-06-03 00:00:00
2018-06-10 00:00:00
2018-06-17 00:00:00
2018-06-24 00:00:00
2018-07-01 00:00:00
2018-07-08 00:00:00
2018-07-15 00:00:00
2018-07-22 00:00:00
2018-07-29 00:00:00
2018-08-05 00:00:00
2018-08-12 00:00:00
2018-08-19 00:00:00
2018-08-26 00:00:00
2018-09-02 00:00:00
2018-09-09 00:00:00
2018-09-16 00:00:00
2018-09-23 00:00:00
2018-09-30 00:00:00
2018-10-07 00:00:00
2018-10-14 00:00:00
2018-10-21 00:00:00
2018-10-28 00:00:00
2018-11-04 00:00:00
2018-11-11 00:00:00
2018-11-18 00:00:00
2018-11-25 00:00:00
2018-12-02 00:00:00
2018-12-09 00:00:00
2018-12-16 00:00:00


In [31]:
price_top10_Market

,Date,top10_Market
0,2018-01-07,"[BTC, XRP, ETH, BCH, ADA, XEM, LTC, TRX, XLM, ..."
1,2018-01-14,"[BTC, ETH, XRP, BCH, ADA, LTC, XEM, XLM, NEO, ..."
2,2018-01-21,"[BTC, ETH, XRP, BCH, ADA, LTC, XEM, NEO, EOS, ..."
3,2018-01-28,"[BTC, ETH, XRP, BCH, ADA, XLM, LTC, NEO, EOS, ..."
4,2018-02-04,"[BTC, ETH, XRP, BCH, ADA, LTC, NEO, XLM, EOS, ..."
...,...,...
151,2020-11-29,"[BTC, ETH, XRP, LINK, BCH, LTC, ADA, BNB, XLM,..."
152,2020-12-06,"[BTC, ETH, XRP, LTC, LINK, BCH, ADA, BNB, XLM,..."
153,2020-12-13,"[BTC, ETH, XRP, LTC, LINK, BCH, ADA, BNB, XLM,..."
154,2020-12-20,"[BTC, ETH, XRP, LTC, BCH, LINK, ADA, BNB, XLM,..."


In [32]:
data_copy_copy = data_copy.copy().reset_index().set_index(['date', 'ticker'])
top10_liquid = pd.DataFrame(columns = ['date', 'top10_Liquid'])

for date in CC100.index:
    date = str(date).split(' ')[0]
    CC_list = data_copy_copy.loc[date].sort_values('volume',ascending= False).iloc[:10].index
    Append = {'date': date, 'top10_Liquid' : CC_list.tolist()}
    top10_liquid = top10_liquid.append(Append, ignore_index=True)

In [33]:
top10_liquid

,date,top10_Liquid
0,2018-01-07,"[BTC , ETH , XRP , TRX , QTUM , BCH , LTC , EO..."
1,2018-01-14,"[BTC , ETH , XRP , ETC , EOS , BCH , LTC , BTG..."
2,2018-01-21,"[BTC , ETH , EOS , XRP , QTUM , ADA , BCH , TR..."
3,2018-01-28,"[BTC , ETH , XRP , EOS , BCH , LTC , ETC , QTU..."
4,2018-02-04,"[BTC , ETH , XRP , LTC , SNT , ADA , BCH , EOS..."
...,...,...
151,2020-11-29,"[BTC , XRP , ETH , LTC , EOS , BCH , ADA , LIN..."
152,2020-12-06,"[BTC , ETH , XRP , LTC , EOS , BCH , LINK , XM..."
153,2020-12-13,"[BTC , ETH , XRP , LTC , BCH , EOS , LINK , XM..."
154,2020-12-20,"[BTC , ETH , XRP , LTC , BCH , EOS , XMR , LIN..."


In [73]:
top10CC = pd.concat([top10_liquid, price_top10_Market], axis=1)
top10CC = top10CC.drop('Date', axis = 1)
top10CC = top10CC.set_index('date')
top10CC

,top10_Liquid,top10_Market
date,,
2018-01-07,"[BTC , ETH , XRP , TRX , QTUM , BCH , LTC , EO...","[BTC, XRP, ETH, BCH, ADA, XEM, LTC, TRX, XLM, ..."
2018-01-14,"[BTC , ETH , XRP , ETC , EOS , BCH , LTC , BTG...","[BTC, ETH, XRP, BCH, ADA, LTC, XEM, XLM, NEO, ..."
2018-01-21,"[BTC , ETH , EOS , XRP , QTUM , ADA , BCH , TR...","[BTC, ETH, XRP, BCH, ADA, LTC, XEM, NEO, EOS, ..."
2018-01-28,"[BTC , ETH , XRP , EOS , BCH , LTC , ETC , QTU...","[BTC, ETH, XRP, BCH, ADA, XLM, LTC, NEO, EOS, ..."
2018-02-04,"[BTC , ETH , XRP , LTC , SNT , ADA , BCH , EOS...","[BTC, ETH, XRP, BCH, ADA, LTC, NEO, XLM, EOS, ..."
...,...,...
2020-11-29,"[BTC , XRP , ETH , LTC , EOS , BCH , ADA , LIN...","[BTC, ETH, XRP, LINK, BCH, LTC, ADA, BNB, XLM,..."
2020-12-06,"[BTC , ETH , XRP , LTC , EOS , BCH , LINK , XM...","[BTC, ETH, XRP, LTC, LINK, BCH, ADA, BNB, XLM,..."
2020-12-13,"[BTC , ETH , XRP , LTC , BCH , EOS , LINK , XM...","[BTC, ETH, XRP, LTC, LINK, BCH, ADA, BNB, XLM,..."


In [74]:
# top10CC['top10_Liquid'] = top10CC['top10_Liquid'].str.replace('[','')
# top10CC['top10_Liquid'] = top10CC['top10_Liquid'].str.replace(']','')
# top10CC['top10_Liquid'] = top10CC['top10_Liquid'].str.replace(' \'','')
# top10CC['top10_Liquid'] = top10CC['top10_Liquid'].str.replace('\'','')

# top10CC['top10_Market'] = str(top10CC['top10_Market']).replace('[','')
# top10CC['top10_Market'] = str(top10CC['top10_Market']).replace(']','')
# top10CC['top10_Market'] = str(top10CC['top10_Market']).replace(' \'','')
# top10CC['top10_Market'] = str(top10CC['top10_Market']).replace('\'','')

# top10CC

In [75]:
top10CC.to_csv('top10CC.csv')